In [1]:
import numpy as np
import pandas as pd
from geopy.geocoders import Nominatim
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import os
import requests, time
import plotly
from plotly_gif import GIF, capture

from import_clean_data import import_clean_airport_df

In [2]:
pd.set_option('display.max_columns', None)

df = import_clean_airport_df('airport_data.csv')

county_df = pd.read_csv('state_and_county_fips_master.csv')

Importing data...
Binarize delay types...
Creating delay size variables...
Getting dummy variables for delay sizes...
Adding other delays...


In [3]:
# Add States
geolocator = Nominatim(user_agent="http")

airports = df[['DEPARTING_AIRPORT', 'LATITUDE', 'LONGITUDE']].drop_duplicates().reset_index(drop=True)

airports_state = airports.apply(lambda col: geolocator.reverse((col.LATITUDE, col.LONGITUDE)).raw['address'], axis=1)

for i in range(airports_state.shape[0]):
    if 'aeroway' in airports_state[i].keys() and airports_state[i]['aeroway'] == 'Bradley International Airport':
        airports_state[i]['ISO3166-2-lvl4'] = 'CT'

airports_col = [ sub['ISO3166-2-lvl4'][-2:] for sub in airports_state ]

airports = airports.reset_index(drop=True)
airports['state'] = airports_col
airports = airports[airports['state'] != 'PR']

df = df.merge(airports[['DEPARTING_AIRPORT', 'state']], on='DEPARTING_AIRPORT')

df = df[df['state']!='PR']

In [5]:
# Add counties

airports_state[33]['county'] = 'New York County'
airports_state[35]['county'] = 'New York County'
airports_state[38]['county'] = 'Denver County'
airports_state[53]['county'] = 'Norfolk County'

airports_col = [ sub['county'] for sub in airports_state if sub['ISO3166-2-lvl4'] != 'US-PR']

airports['county'] = airports_col

df = df.merge(airports[['DEPARTING_AIRPORT', 'county']], on='DEPARTING_AIRPORT')

In [6]:
fips_df = df.merge(county_df, left_on=['county', 'state'], right_on=['name', 'state'], how='left')

fips_df.loc[fips_df['DEPARTING_AIRPORT']=='Lambert-St. Louis International', 'fips'] = 29510
fips_df.loc[fips_df['DEPARTING_AIRPORT']=='Bradley International', 'fips'] = 9003
fips_df.loc[fips_df['DEPARTING_AIRPORT']=='Norfolk International', 'fips'] = 51710

fips_df = fips_df[~fips_df['fips'].isna()]#.DEPARTING_AIRPORT.value_counts()

In [7]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [11]:
df_prov = fips_df.groupby(['county','fips', 'MONTH']).WEATHER_DELAY.mean().reset_index()

fig = px.scatter_geo(df_prov,
                           geojson=counties, locations='fips', color='WEATHER_DELAY',
                           color_continuous_scale="Viridis_r",
                           range_color=(0, 0.065),
                           scope="usa",
                           labels={'WEATHER_DELAY':'Perc of Weather Delay'},
                           animation_frame='MONTH', 
                           size=df_prov["WEATHER_DELAY"]*100,
                           title='Weather Delays for US Regions'
                          )
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

In [10]:
df_prov = fips_df.groupby(['county','fips', 'MONTH']).DEP_DEL15.mean().reset_index()

fig = px.scatter_geo(df_prov,
                           geojson=counties, locations='fips', color='DEP_DEL15',
                           color_continuous_scale="Viridis_r",
                           range_color=(0.067, 0.354),
                           scope="usa",
                           labels={'DEP_DEL15':'Perc of Delays'},
                           animation_frame='MONTH', 
                           size=df_prov["DEP_DEL15"]*100,
                           title='Delays for US Regions'
                          )
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})

In [ ]:
import PIL
import io

frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png"))))
    
# create animated GIF
frames[0].save(
        "AIRPORT_2.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=500,
        loop=0,
    )